In [ ]:
import pandas as pd

import json
from tqdm.auto import tqdm 

def read_first_n_lines(file_path, n=10000):
    data = []
    with open(file_path, 'r') as file:
        for i, line in tqdm(enumerate(file), total=n):
            if i >= n:
                break
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return pd.DataFrame(data)

In [ ]:
df = read_first_n_lines('data/arxiv-metadata-oai-snapshot.json', 1000)

#new
df = df.drop_duplicates(subset='title', keep='first')
df = df.reset_index(drop=True)

In [ ]:
import spacy
from tqdm.auto import tqdm

nlp = spacy.load("en_core_web_sm")
def lemma(docs=df.abstract):
    pip = nlp.pipe(docs, batch_size=32, n_process=1, disable=["parser", "ner"])
    return [
        [tok.lemma_ for tok in doc if not tok.is_punct and not tok.is_space]
        for doc in tqdm(pip, total=len(docs))
    ]

import os
df['abstract'] = df['title'] +' '+df['abstract']
documents = lemma(df.title)


In [ ]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary


dictionary = Dictionary(documents)


corpus = [dictionary.doc2bow(doc) for doc in documents]


tfidf_model = TfidfModel(corpus)


tfidf_corpus = tfidf_model[corpus]



In [ ]:
import pickle 

with open('tfidf_corpus (1).pkl', 'wb') as f:
        pickle.dump({'ids':df['id'].tolist(), 'vectors': tfidf_corpus},f)